In [1]:
import pandas as pd
import numpy as np

from m4_feats_functions import *
from m5_sb_models import *
from m7_utils import *
from m3_model_params import lgb_params_1, xgb_params
from sklearn.metrics import mean_squared_error
from tqdm import tqdm  # Import tqdm

import os, warnings
warnings.filterwarnings('ignore')

In [2]:
# Paths to the train and test directories
INPUT_DIR = 'kaggle/input/linking-writing-processes-to-writing-quality'
FEATURE_STORE = 'feature_store'
train_scores = pd.read_csv(f'{INPUT_DIR}/train_scores.csv')
train_feats = pd.read_pickle('feature_selection/train_feats.pkl')
# train_feats = train_feats.merge(train_scores, on='id', how='left')
test_feats = pd.read_pickle('feature_selection/test_feats.pkl')
train_dir = 'feature_store/train'
test_dir = 'feature_store/test'
# Usage
seed = 42
n_repeats = 5
n_splits = 10
target_col = 'score'

In [3]:
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold
import random

def random_param_search(train_feats, test_feats, n_iterations):
    # Define parameter ranges
    param_ranges = {
        'boosting_type': ['gbdt'],
        'num_leaves': (15, 150),  # Range for num_leaves
        'learning_rate': (0.005, 0.2),  # Range for learning_rate
        'max_depth': (5, 20),  # Range for max_depth
        'min_child_samples': (5, 50),  # Range for min_child_samples
        'reg_alpha': (0, 1),  # Range for L1 regularization
        'reg_lambda': (0, 1),  # Range for L2 regularization
        'n_estimators': (100, 500),  # Range for L2 regularization
    }

    best_rmse = float("inf")
    best_params = None
    test_predictions_df = pd.DataFrame()

    for _ in range(n_iterations):

        # Sampling parameters from ranges
        params = {
            'boosting_type': random.choice(param_ranges['boosting_type']),
            'num_leaves': random.randint(*param_ranges['num_leaves']),
            'learning_rate': random.uniform(*param_ranges['learning_rate']),
            'max_depth': random.randint(*param_ranges['max_depth']),
            'min_child_samples': random.randint(*param_ranges['min_child_samples']),
            'reg_alpha': random.uniform(*param_ranges['reg_alpha']),
            'reg_lambda': random.uniform(*param_ranges['reg_lambda']),
            'n_estimators': random.randint(*param_ranges['n_estimators']),
            'verbose': -1
        }
        
        # Train and evaluate model
        _, _, rmse, model = lgb_pipeline(train_feats, test_feats, params)
                                        
        new_iter = {'rmse': [rmse],
                    'params': str(model.get_params())}

        test_predictions_df = pd.concat([test_predictions_df, pd.DataFrame(data=new_iter)])
        # Update best parameters if current model is better
        if rmse < best_rmse:
            print(f"New best RMSE: {rmse:.6f}")
            best_rmse = rmse
            best_params = model.get_params()
    
    print(f"Best RMSE: {best_rmse:.6f}")
    print("Best Parameters:", best_params)
    return test_predictions_df

# Example usage
test_predictions_df = random_param_search(train_feats, test_feats, 500)
test_predictions_df.sort_values('rmse').to_csv('params.csv')

Final RMSE over 50: 0.618084. Std 0.8397
RMSE by fold 0.617989. Std 0.0112
New best RMSE: 0.618084
Final RMSE over 50: 0.626774. Std 0.8405
RMSE by fold 0.626670. Std 0.0119
Final RMSE over 50: 0.628651. Std 0.8529
RMSE by fold 0.628551. Std 0.0117
Final RMSE over 50: 0.635543. Std 0.8381
RMSE by fold 0.635409. Std 0.0136
Final RMSE over 50: 0.646778. Std 0.6463
RMSE by fold 0.646724. Std 0.0088
Final RMSE over 50: 0.626005. Std 0.8499
RMSE by fold 0.625916. Std 0.0110
Final RMSE over 50: 0.637574. Std 0.8624
RMSE by fold 0.637448. Std 0.0131
Final RMSE over 50: 0.623872. Std 0.8477
RMSE by fold 0.623729. Std 0.0137
Final RMSE over 50: 0.625047. Std 0.8453
RMSE by fold 0.624935. Std 0.0123
Final RMSE over 50: 0.631021. Std 0.8565
RMSE by fold 0.630960. Std 0.0092
Final RMSE over 50: 0.627227. Std 0.8365
RMSE by fold 0.627084. Std 0.0139
Final RMSE over 50: 0.610570. Std 0.8246
RMSE by fold 0.610482. Std 0.0108
New best RMSE: 0.610570
Final RMSE over 50: 0.633261. Std 0.8550
RMSE by fol

In [3]:
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold
import random

def random_param_search_xgb(train_feats, test_feats, n_iterations):
    # Define parameter ranges
    param_ranges = {
        'booster': ['gbtree'],
        'max_depth': (3, 10),  # Range for max_depth
        'learning_rate': (0.01, 0.2),  # Range for learning_rate
        'min_child_weight': (1, 6),  # Range for min_child_weight
        'gamma': (0, 0.5),  # Range for gamma
        'subsample': (0.5, 1),  # Range for subsample
        'colsample_bytree': (0.5, 1),  # Range for colsample_bytree
    }

    best_rmse = float("inf")
    best_params = None
    test_predictions_df = pd.DataFrame()

    for _ in range(n_iterations):

        # Sampling parameters from ranges
        params = {
            'booster': random.choice(param_ranges['booster']),
            'max_depth': random.randint(*param_ranges['max_depth']),
            'learning_rate': random.uniform(*param_ranges['learning_rate']),
            'min_child_weight': random.randint(*param_ranges['min_child_weight']),
            'gamma': random.uniform(*param_ranges['gamma']),
            'subsample': random.uniform(*param_ranges['subsample']),
            'colsample_bytree': random.uniform(*param_ranges['colsample_bytree']),
            'device': 'cuda',
        }
        
        # Train and evaluate model
        booster = params['booster']
        _, _, rmse, model = xgb_cv_pipeline(train_feats, test_feats, params)
                                        
        new_iter = {'rmse': [rmse],
                    'params': str(model.get_params())}

        test_predictions_df = pd.concat([test_predictions_df, pd.DataFrame(data=new_iter)])
        # Update best parameters if current model is better
        if rmse < best_rmse:
            print(f"New best RMSE: {rmse:.6f}")
            best_rmse = rmse
            best_params = model.get_params()
    
    print(f"Best RMSE: {best_rmse:.6f}")
    print("Best Parameters:", best_params)
    return test_predictions_df

# Example usage
test_predictions_df_xgb = random_param_search_xgb(train_feats, test_feats, 300)
test_predictions_df_xgb.sort_values('rmse').to_csv('params_xgb.csv', index=False)

Iterations:   0%|          | 0/5 [00:00<?, ?it/s]

/root/miniconda3/envs/lrp/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [01:11:24] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
Iterations: 100%|██████████| 5/5 [01:00<00:00, 12.13s/it]


XGB Average RMSE over 50 folds: 0.605564
New best RMSE: 0.605564


Iterations: 100%|██████████| 5/5 [00:45<00:00,  9.04s/it]


XGB Average RMSE over 50 folds: 0.610621


...

XGB Average RMSE over 50 folds: 0.610296


Iterations: 100%|██████████| 5/5 [01:04<00:00, 12.85s/it]


XGB Average RMSE over 50 folds: 0.606215


Iterations: 100%|██████████| 5/5 [01:12<00:00, 14.53s/it]

XGB Average RMSE over 50 folds: 0.615773
Best RMSE: 0.601441
Best Parameters: {'objective': 'reg:squarederror', 'base_score': None, 'booster': 'gbtree', 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': 0.8739265814988941, 'device': 'cuda', 'early_stopping_rounds': 250, 'enable_categorical': False, 'eval_metric': None, 'feature_types': None, 'gamma': 0.3893408430829386, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': 0.18205557831997368, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': 3, 'max_leaves': None, 'min_child_weight': 4, 'missing': nan, 'monotone_constraints': None, 'multi_strategy': None, 'n_estimators': None, 'n_jobs': None, 'num_parallel_tree': None, 'random_state': 42, 'reg_alpha': None, 'reg_lambda': None, 'sampling_method': None, 'scale_pos_weight': None, 'subsample': 0.6407940935121966, 'tree_method': None, 'validate_param

In [4]:
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold
import random

def random_param_search_xgb(train_feats, test_feats, n_iterations):
    # Define parameter ranges
    param_ranges = {
        'booster': ['gbtree'],
        'max_depth': (3, 10),  # Range for max_depth
        'learning_rate': (0.01, 0.2),  # Range for learning_rate
        'min_child_weight': (1, 6),  # Range for min_child_weight
        'gamma': (0, 0.5),  # Range for gamma
        'subsample': (0.5, 1),  # Range for subsample
        'colsample_bytree': (0.5, 1),  # Range for colsample_bytree
    }

    best_rmse = float("inf")
    best_params = None
    test_predictions_df = pd.DataFrame()

    for _ in range(n_iterations):

        # Sampling parameters from ranges
        params = {
            'booster': random.choice(param_ranges['booster']),
            'max_depth': random.randint(*param_ranges['max_depth']),
            'learning_rate': random.uniform(*param_ranges['learning_rate']),
            'min_child_weight': random.randint(*param_ranges['min_child_weight']),
            'gamma': random.uniform(*param_ranges['gamma']),
            'subsample': random.uniform(*param_ranges['subsample']),
            'colsample_bytree': random.uniform(*param_ranges['colsample_bytree']),
            'device': 'cuda',
        }
        
        # Train and evaluate model
        booster = params['booster']
        _, _, rmse, model = xgb_cv_pipeline(train_feats, test_feats, params)
                                        
        new_iter = {'rmse': [rmse],
                    'params': str(model.get_params())}

        test_predictions_df = pd.concat([test_predictions_df, pd.DataFrame(data=new_iter)])
        # Update best parameters if current model is better
        if rmse < best_rmse:
            print(f"New best RMSE: {rmse:.6f}")
            best_rmse = rmse
            best_params = model.get_params()
    
    print(f"Best RMSE: {best_rmse:.6f}")
    print("Best Parameters:", best_params)
    return test_predictions_df

# Example usage
test_predictions_df_xgb = random_param_search_xgb(train_feats, test_feats, 850)
test_predictions_df_xgb.sort_values('rmse').to_csv('params_xgb2.csv', index=False)

Iterations:   0%|          | 0/5 [00:00<?, ?it/s]

Iterations: 100%|██████████| 5/5 [01:37<00:00, 19.58s/it]


XGB Average RMSE over 50 folds: 0.609284
New best RMSE: 0.609284


Iterations: 100%|██████████| 5/5 [02:32<00:00, 30.53s/it]


XGB Average RMSE over 50 folds: 0.615890


...

XGB Average RMSE over 50 folds: 0.612679


Iterations: 100%|██████████| 5/5 [00:47<00:00,  9.56s/it]


XGB Average RMSE over 50 folds: 0.605320


Iterations:  40%|████      | 2/5 [02:32<03:49, 76.41s/it]


KeyboardInterrupt: 

In [ ]:
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold
import random

def random_param_search_xgb(train_feats, test_feats, n_iterations):
    # Define parameter ranges
    param_ranges = {
        'booster': ['gbtree', 'gblinear'],
        'max_depth': (3, 10),  # Range for max_depth
        'learning_rate': (0.01, 0.2),  # Range for learning_rate
        'n_estimators': (100, 1000),  # Range for n_estimators
        'min_child_weight': (1, 6),  # Range for min_child_weight
        'gamma': (0, 0.5),  # Range for gamma
        'subsample': (0.5, 1),  # Range for subsample
        'colsample_bytree': (0.5, 1),  # Range for colsample_bytree
    }

    best_rmse = float("inf")
    best_params = None
    test_predictions_df = pd.DataFrame()

    for _ in range(n_iterations):

        # Sampling parameters from ranges
        params = {
            'booster': random.choice(param_ranges['booster']),
            'max_depth': random.randint(*param_ranges['max_depth']),
            'learning_rate': random.uniform(*param_ranges['learning_rate']),
            'n_estimators': random.randint(*param_ranges['n_estimators']),
            'min_child_weight': random.randint(*param_ranges['min_child_weight']),
            'gamma': random.uniform(*param_ranges['gamma']),
            'subsample': random.uniform(*param_ranges['subsample']),
            'colsample_bytree': random.uniform(*param_ranges['colsample_bytree']),
            'device': 'cuda',
            'tree_method': 'hist',
        }
        
        # Train and evaluate model
        booster = params['booster']
        _, _, rmse, model = xgb_cv_pipeline(train_feats, test_feats, params, booster)
                                        
        new_iter = {'rmse': [rmse],
                    'params': str(model.get_params())}

        test_predictions_df = pd.concat([test_predictions_df, pd.DataFrame(data=new_iter)])
        # Update best parameters if current model is better
        if rmse < best_rmse:
            print(f"New best RMSE: {rmse:.6f}")
            best_rmse = rmse
            best_params = model.get_params()
    
    print(f"Best RMSE: {best_rmse:.6f}")
    print("Best Parameters:", best_params)
    return test_predictions_df

# Example usage
test_predictions_df_xgb = random_param_search_xgb(train_feats, test_feats, 300)
test_predictions_df_xgb.sort_values('rmse').to_csv('params_xgb2.csv', index=False)

In [14]:
test_predictions_df.sort_values('rmse')

In [21]:
test_predictions_df['params'].head(3)

0    {'boosting_type': 'gbdt', 'class_weight': None...
0    {'boosting_type': 'dart', 'class_weight': None...
0    {'boosting_type': 'gbdt', 'class_weight': None...
Name: params, dtype: object